In [1]:
#Import Dependancies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Open Driver
executable_path = {'executable_path': 'ChromeDriver/chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
url = 'https://www.alirts.oshpd.ca.gov/default.aspx'
browser.visit(url)

In [4]:
#Seach by OSHPD ID. Need to find the list of all reports by OSHPD ID. Elizabeth = 406374502, Lightbridge = 406374352
browser.fill('BasicSearch1:txtSearch', '406374352')

# Find and click the 'search' button
button = browser.find_by_name('BasicSearch1:btnSearch').first.click()
browser.click_link_by_partial_href('UtilReports')

#Grab data and set it equal to soup, we will go through the soup to find the last OSHPD Report link
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
#Grab the link to the last OSHPD report in the list
reports = soup.find_all('a', text='View')
link = reports[-1]['href']
browser.visit('https://www.alirts.oshpd.ca.gov/' + link)

#Visit the OSHPD report. Set the HTML equal to oshpdsoup to parse the data
oshpdhtml = browser.html
oshpdsoup = BeautifulSoup(oshpdhtml, 'html.parser')

In [6]:
#Grab all tables inside of oshpdsoup and initialize variables
table = oshpdsoup.find_all('tbody')
tablextable = []
x = 0

#Loop through all the tables grabbed and check to see if they have a section body and an inner table. If so, keep them
#inside of new variable tablextable. If not, drop them and print that the table has been dropped
for table in table:
    headercheck = table.find_all('td', class_='blackheader')
    bodycheck = table.find_all('tbody') 
    try:
        if headercheck[0]:
            try:
                if bodycheck[0]:
                    tablextable.append(table)
            except:
                print(f"Table {x} dropped")
    except:
        print(f"Table {x} dropped")
    x += 1

#Drop off first two tables because they contain the table of contents.
drop = tablextable.pop(0)
print()
print('Table 0 dropped')
print()

drop = tablextable.pop(0)
print('Table 4 dropped')
print()

Table 1 dropped
Table 2 dropped
Table 3 dropped
Table 5 dropped
Table 6 dropped
Table 7 dropped
Table 9 dropped
Table 11 dropped
Table 12 dropped
Table 13 dropped
Table 14 dropped
Table 15 dropped
Table 16 dropped
Table 17 dropped
Table 18 dropped
Table 19 dropped
Table 20 dropped
Table 21 dropped
Table 23 dropped
Table 24 dropped
Table 25 dropped
Table 26 dropped
Table 27 dropped
Table 28 dropped
Table 30 dropped
Table 31 dropped
Table 32 dropped
Table 33 dropped
Table 35 dropped
Table 36 dropped
Table 37 dropped
Table 38 dropped
Table 39 dropped
Table 40 dropped
Table 42 dropped
Table 43 dropped
Table 44 dropped
Table 45 dropped
Table 46 dropped
Table 48 dropped
Table 49 dropped
Table 50 dropped
Table 51 dropped
Table 52 dropped
Table 53 dropped
Table 54 dropped
Table 55 dropped
Table 56 dropped
Table 58 dropped
Table 59 dropped
Table 60 dropped
Table 62 dropped
Table 63 dropped
Table 64 dropped
Table 66 dropped
Table 67 dropped
Table 68 dropped
Table 70 dropped
Table 71 dropped
Tabl

In [7]:
#Drop tables for home health information that isn't used in this report
drop = tablextable.pop(1)
drop = tablextable.pop(1)
drop = tablextable.pop(1)
drop = tablextable.pop(7)

print("Table 1, 2, 3 and 10 dropped")

Table 1, 2, 3 and 10 dropped


In [8]:
#Check what tables are remaining
for table in tablextable:
    header = table.find('td', class_='blackheader').text.strip()
    print(header)
    print('----------------------')

Section 1 - General Information
----------------------
Section 5 - Hospice Description
----------------------
Section 6 - Hospice Services
----------------------
Section 7 - Hospice Patient Information
----------------------
Section 8 - Hospice Utilization
----------------------
Section 9 - Hospice Care and Source of Payment
----------------------
Section 10 - Hospice Income and Expenses Statement
----------------------


In [9]:
#For the first section, the layout is different from the rest. The data is stored in content/subtitle
#For this, we try to print out the data fields that we think is there. If there is an error, we will pass
#For the facility name, the name is stored in the subtitle class
data = {}
for tablerow in tablextable[0].find_all('tr'):
    content = tablerow.find_all('td', class_='content')
    subtitle = tablerow.find_all('td', class_='subtitle')
    
    try:
        try:
            print(content[0].text.strip())
            print(subtitle[0].text.strip())
            print(content[1].text.strip())

        except:
            pass
            
        print(subtitle[1].text.strip())
        
    except:
        pass
    print('-------------')

-------------
-------------
1.
Facility Name:
LIGHTBRIDGE HOSPICE
-------------
2.
OSHPD ID Number:
406374352
-------------
3.
Street Address:
6155 CORNERSTONE COURT E
-------------


STE 220
-------------
4.
City:
SAN DIEGO
-------------
5.
Zip:
92121
-------------
6.
Facility Phone No.:
(
			858) 
			458 - 
			2992  
			ext. 
			209
-------------
7.
Administrator Name:
Pamela Hough
-------------
9.
Was this agency in operation at any time during the year?:
Yes
-------------
10.
Operation Open From:
1/1/2017
-------------
11.
Operation Open To:
12/31/2017
-------------
12.
Name of Parent Corporation:
LightBridge Hospice, LLC dba LightBridge Hospice and Palliative Care
-------------
13.
Corporate Business Address:
6155 Cornerstone Court East  
			Suite 220
-------------
14.
City:
San Diego
-------------
15.
State:
CA
-------------
16.
Zip:
92121
			-
			4737
-------------
17.
Person Completing Report:
Siva Cameron
-------------
18.
Phone No.:
858-458-2992
-------------
19.
Fax No.:
858

In [14]:
#For the main data section, we loop through each section. In each section we grab the name of the section,
#and all the tables inside. We loop through all of the tables and find the rows
#The first column in each row (line no) is the key to the values inside of each row
#Our final structure is like this: Section:Line No:Values
for section in tablextable[1:]:
    name = section.find('td', class_='blackheader').text.strip()
    temptable = {}
    
    for table in section.find_all('table'):
        
        for row in table.find_all('tr'):
            rowdata1 = row.find('td', class_='content')

            try:
                rowdata2 = row.find('td', class_='content').find_next_siblings()
                temprow = []
                for data2 in rowdata2:
                    temprow.append(data2.text.strip())
                temptable[rowdata1.text.strip()] = temprow

            except:
                pass
    data[name] = temptable

In [33]:
#for section in list(data.keys()):
#for listno in list(data[section].keys()):
#df = pd.DataFrame.from_dict(data['Section 5 - Hospice Description'])
print(data['Section 5 - Hospice Description'])

{'1.': ['Select the one category that best describes the licensee type of control of your hospice from drop down list:', 'Investor - Limited Liability Company'], '5.': ['Medicare and Medi-Cal'], '10.': ['Accredited by ACHC', 'None'], '11.': ['Accredited by CHAP', 'None'], '12.': ['Accredited by JCAHO', 'Accredited'], '13.': ['Accredited by other', 'None'], '20.': ['Free Standing'], '25.': ['Mixed Urban and Rural']}


In [24]:
data

{'Section 5 - Hospice Description': {'1.': ['Select the one category that best describes the licensee type of control of your hospice from drop down list:',
   'Investor - Limited Liability Company'],
  '5.': ['Medicare and Medi-Cal'],
  '10.': ['Accredited by ACHC', 'None'],
  '11.': ['Accredited by CHAP', 'None'],
  '12.': ['Accredited by JCAHO', 'Accredited'],
  '13.': ['Accredited by other', 'None'],
  '20.': ['Free Standing'],
  '25.': ['Mixed Urban and Rural']},
 'Section 6 - Hospice Services': {'1.': ['Survivors of hospice patients',
   '538'],
  '2.': ['Survivors of persons not receiving hospice care', '0'],
  '3.': ['Patient / Family Services', '2,133'],
  '4.': ['Bereavement', '66'],
  '5.': ['Administrative', '1,228'],
  '6.': ['Medicare Reportable Hours(sum lines 3-5)', '3,427'],
  '7.': ['Fundraising', '0'],
  '9.': ['Other', '0'],
  '10.': ['Total', '3,427'],
  '11.': ['Hospice Inpatient Facility / Unit', 'No'],
  '12.': ['Specialized Pediatric Program', 'No'],
  '13.': [